# Libraries

In [ ]:
from PIL import Image
import os
import numpy as np
from sklearn.metrics import confusion_matrix
import tensorflow as tf
tf.get_logger().setLevel(3)

# Building the Neural Network

In [ ]:
def create_cnn(width=154, height=154, depth=3, filters=(16, 32, 64)):
    inputShape = (height, width, depth)
    chanDim = -1

    inputs = tf.keras.Input(shape=inputShape)
	    # loop over the number of filters
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
            # CONV => RELU => BN => POOL
        x = tf.keras.layers.Conv2D(f, (3, 3), padding="same")(x)
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(16)(x)
    x = tf.keras.layers.Activation("relu")(x)
    x = tf.keras.layers.BatchNormalization(axis=chanDim)(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(11)(x)
    x = tf.keras.layers.Activation("softmax")(x)
    # construct the CNN
    model = tf.keras.Model(inputs, x)
    # return the CNN
    return model    

# Preprocessing and converting the data to train and test

In [ ]:
data_path = 'dataset/'

X_train = []
X_test = []
Y_train = []
Y_test = []

for dir in os.listdir(data_path):
    for classes in os.listdir(data_path+'/'+ dir):
        for file in os.listdir(data_path+'/'+dir+'/'+classes):
            fmt = file.split('.')

            if fmt[-1] == "jpg":
                img_path = data_path+'/'+dir+'/'+classes+'/'+file
                img = np.array(Image.open(img_path))
                if dir == 'train':
                    X_train.append(img)
                    Y_train.append(int(classes))
                elif dir == 'test':
                    X_test.append(img)
                    Y_test.append(int(classes))

X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

Y = np.zeros((11, len(Y_train)))
for i in range(0,len(Y_train)):
    Y[Y_train[i]//10,i] = 1

# Training the model

In [ ]:
model = create_cnn(154, 154, 3)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x=X_train, y=Y.T, epochs=200, batch_size=5, verbose=0)

# Testing the trained model

In [6]:
preds = model.predict(X_test)
y_pred = np.zeros(len(preds))

for i in range(0,len(preds)):
    y_pred[i] = 10 * np.argmax(preds[i,:])

print("True values: ", Y_test)
print("Predicted values:", y_pred)
print("Confusion Matrix :")
print(confusion_matrix(Y_test, y_pred))

1/1 [==============================] - 0s 24ms/step
True values:  [ 80  70  50  10  40 100  90   0  60  20  30]
Predicted values: [ 70.  70.  50.  10.  40. 100.  80.   0.  60.  20.  30.]
Confusion Matrix :
[[1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1]]
